<a href="https://colab.research.google.com/github/reyaayah/Music-Recommendation-System/blob/main/MRS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("/content/drive/MyDrive/mrs/spotify_millsongdata.csv.zip")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df.shape

(57650, 4)

In [ ]:
df= df.sample(5000).drop('link',axis=1).reset_index(drop=True)

In [ ]:
df.head(10)

,artist,song,text
0,Opeth,The Funeral Portrait,You wait by the window \r\nMorning's breath o...
1,Whitney Houston,Where You Are,I saw the news this morning \r\nAnd saw your ...
2,Ed Sheeran,Give Me Love,"Give me love like her, \r\n'Cause lately I've..."
3,Celine Dion,Everybody's Talkin' My Baby Down,People are sayin' \r\nThat boy is gonna hurt ...
4,Pet Shop Boys,Left To My Own Devices,I get out of bed at half past ten \r\nPhone u...
5,Planetshakers,Speak To Me Lord,I look to You my God \r\nAnd lift my eyes to ...
6,Def Leppard,Animal,"A wild ride, over stony ground \r\nSuch a lus..."
7,Widespread Panic,Action Man,"Willie said ""he was the mostest horse"" \r\nA ..."
8,Frank Zappa,Cocaine Decisions,Chop a line now \r\nCocaine decisions \r\nYo...
9,Radiohead,How Do You?,"He's bitter and twisted, he knows what he want..."


In [ ]:
df.shape

(5000, 3)

In [ ]:
df['text']=df['text'].str.lower().replace(r'^\w\s',' ').replace(r'\n',' ',regex=True)

In [ ]:
df.tail(5)

,artist,song,text
4995,James Taylor,First Of May,and the light between us \r which we could no...
4996,Frank Sinatra,Bonita,"what can i say to you, bonita, what magic word..."
4997,Johnny Cash,Big Iron,to the town of agua fria rode a stranger one f...
4998,Dusty Springfield,A Song For You,i've been so many places \r in my life and ti...
4999,Kris Kristofferson,Why Me,why me lord what have i ever done to deserve e...


In [ ]:

from nltk.stem.porter import PorterStemmer

In [ ]:
!pip install nltk
import nltk
nltk.download('punkt')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
stemmer=PorterStemmer()

In [ ]:
def token(txt):
  token =nltk.word_tokenize(txt)
  a=[stemmer.stem(w) for w in token]
  return " ".join(a)

In [ ]:
token("You are beautiful, beauty")

'you are beauti , beauti'

In [ ]:
df['text'].apply(lambda x: token(x))

0       you wait by the window morn 's breath on the s...
1       i saw the news thi morn and saw your face acro...
2       give me love like her , 'caus late i 've been ...
3       peopl are sayin ' that boy is gon na hurt you ...
4       i get out of bed at half past ten phone up a f...
                              ...                        
4995    and the light between us which we could not qu...
4996    what can i say to you , bonita , what magic wo...
4997    to the town of agua fria rode a stranger one f...
4998    i 've been so mani place in my life and time i...
4999    whi me lord what have i ever done to deserv ev...
Name: text, Length: 5000, dtype: object

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
tfid=TfidfVectorizer(analyzer='word',stop_words='english')

In [ ]:
matrix=tfid.fit_transform(df['text'])

In [ ]:
similer=cosine_similarity(matrix)

In [ ]:
similer[0]

array([1.00000000e+00, 2.68195715e-02, 2.61043620e-02, ...,
       2.55054870e-02, 7.74527467e-04, 7.10379361e-03])

In [ ]:
df[df['song']=='First Of May'].index[0]

4995

In [ ]:
def recommender(song_name):
  idx=df[df['song']==song_name].index[0]
  distance=sorted(list(enumerate(similer[idx])),reverse=True,key=lambda x:x[1])
  song=[]
  for s_id in distance[1:21]:
    song.append(df.iloc[s_id[0]].song)
  return song

In [ ]:
recommender('First Of May')

['Aztec Priests Song',
 'Dj Seduces Sonny',
 "He Ain't Give You None",
 "Stayin' Alive",
 "Beegee's Medley",
 'One And Only',
 'Shut Up And Make Love',
 'Keep Smiling',
 'This Time',
 'Angels With Dirty Faces',
 'High Time',
 'Port Arthur High School Reunion',
 'Fire',
 "Blazin'",
 'Rise Above',
 'Schizo',
 'Have A Banana!',
 'Mississippi Queen',
 "Learning How To Rock'n'roll",
 '634-5789']

In [ ]:
import pickle

In [ ]:
pickle.dump(similer,open("similarity","wb"))

In [ ]:
pickle.dump(df,open("df","wb"))